# Imports

In [ ]:
import pandas as pd
import boto3
import json
import configparser

In [ ]:
from IPython.display import JSON

In [ ]:
import getpass

# Configuration variables

In [ ]:
# AWS credentials
AWS_KEY                = None
AWS_SECRET             = None
AWS_REGION             = None

# Cluster configuration
DWH_CLUSTER_TYPE       = None
DWH_NUM_NODES          = None
DWH_NODE_TYPE          = None

# Redshift details
DWH_CLUSTER_IDENTIFIER = None
DWH_DB                 = None
DWH_DB_USER            = None
DWH_DB_PASSWORD        = None
DWH_PORT               = None

# IAM role
DWH_IAM_ROLE_NAME      = None

In [ ]:
AWS_SECRET = getpass.getpass("Please give your secret key")

# Parse config file

In [ ]:
config = configparser.ConfigParser()

with open('/home/workspace/dev/dwh.cfg') as fp:
    config.read_file(fp)
    AWS_KEY                = config.get('AWS','KEY')
#     AWS_SECRET             = config.get('AWS','SECRET')
    AWS_REGION             = config.get('AWS','REGION')
    DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
    DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
    DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")
    DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
    DWH_DB                 = config.get("DWH","DWH_DB")
    DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
    DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
    DWH_PORT               = config.get("DWH","DWH_PORT")
    DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

# Create IAM Role 

In [ ]:
from botocore.exceptions import ClientError

In [ ]:
aws_iam = boto3.client(
    'iam',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET,
)

In [ ]:
try:
    resp = aws_iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps({
            'Statement': [{
                'Action': 'sts:AssumeRole',
                'Effect': 'Allow',
                'Principal': {
                    'Service': 'redshift.amazonaws.com'
                }
            }],
            'Version': '2012-10-17'
        })
    )
except Exception as err:
    print(err)

In [ ]:
resp = aws_iam.attach_role_policy(
    RoleName=DWH_IAM_ROLE_NAME,
    PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
)['ResponseMetadata']['HTTPStatusCode']

In [ ]:
print(f"[IAM - Attach Role Policy] {resp}")

In [ ]:
roleArn = aws_iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

In [ ]:
print(f"[IAM - Role ARN] {roleArn}")

# Create Redshift cluster

In [ ]:
aws_redshift = boto3.client(
    'redshift',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET
)

In [ ]:
try:
    resp = aws_redshift.create_cluster(
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        IamRoles=[roleArn]
    )
except Exception as err:
    print(err)

In [ ]:
clusterProps = aws_redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

In [ ]:
JSON(clusterProps, expanded=False)

# Create an EC2 instance 

In [ ]:
DWH_ENDPOINT = clusterProps['Endpoint']['Address']
DWH_ROLE_ARN = clusterProps['IamRoles'][0]['IamRoleArn']

In [ ]:
print(f"Endpoint: {DWH_ENDPOINT}")

In [ ]:
print(f"Role ARN: {DWH_ROLE_ARN}")

In [ ]:
aws_ec2 = boto3.resource(
    'ec2',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET
)

In [ ]:
try:
    vpc = aws_ec2.Vpc(id=clusterProps['VpcId'])
    
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as err:
    print(err)

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)

In [ ]:
%sql $conn_string